In [1]:
from IPython.core.display import TextDisplayObject
import pandas as pd
import matplotlib.pyplot as mp
import numpy as np
import csv
from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path
import os
import math

In [2]:
def remove_first(x):
  x =  x[1:]
  x = float(x)
  return x

def remove_last(x):
  x = x[:(len(x)-1)]
  x = float(x)
  return x

def revalue(x):
  if x >180:
    x= -(360-x)
  return x

In [3]:
def process_data(filepath):
  
  b = pd.read_csv(filepath)

  b = b.iloc[3:]
  b = b.reset_index(drop=True)

  b.rename(columns={'Lapsed Time': 'Time'}, inplace=True)
  b.rename(columns={'Target Name': 'Target_Name'}, inplace=True)

  b['Time_Diff'] = b['Time'].diff()
  b['Time_Diff'] = b['Time_Diff'].apply(lambda x: max(0, x))

  b['X Euler Angle'] = b['X Euler Angle'].apply(remove_first)
  b['Z Euler Angle'] = b['Z Euler Angle'].apply(remove_last)

  b['X_A'] = b['X Euler Angle']
  b['Y_A'] = b['Y Euler Angle']
  b['Z_A'] = b['Z Euler Angle']

  b['X_A_Rev'] = b['X Euler Angle'].apply(revalue)
  b['Y_A_Rev'] = b['Y Euler Angle'].apply(revalue)
  b['Z_A_Rev'] = b['Z Euler Angle'].apply(revalue)

  b['X_A_Rev_Diff'] = abs(b['X_A_Rev'].diff())
  b['Y_A_Rev_Diff'] = abs(b['Y_A_Rev'].diff())
  b['Z_A_Rev_Diff'] = abs(b['Z_A_Rev'].diff())

  # Removing all rows that say Mission Complete
  label_to_remove = 'Mission complete'
  b = b[b['Target_Name'] != label_to_remove]

  b.fillna(0, inplace=True)

  b_df = b[['Target_Name', 'X', 'Z', 'X_A', 'X_A_Rev', 'X_A_Rev_Diff', 'Y_A', 'Y_A_Rev', 'Y_A_Rev_Diff', 'Z_A', 'Z_A_Rev', 'Z_A_Rev_Diff', 'Time', 'Time_Diff']].copy()
  
  return b_df 

In [4]:
# Setting filepath to all data folders
%store -r fp_folder
os.makedirs(fp_folder+'Target_Data/', exist_ok=True)

sub_folders = []
for i in os.listdir(fp_folder):
    if os.path.isdir(os.path.join(fp_folder, i)) and (i.startswith('BNC') or i.startswith('NAV')):
        sub_folders.append(i)

num_subjects = len(sub_folders)
sub_folders = sorted(sub_folders)
print(sub_folders)

['BNC01', 'BNC02', 'BNC03', 'BNC04', 'BNC05', 'BNC07', 'BNC08', 'BNC09', 'BNC11', 'BNC12', 'BNC23', 'BNC26', 'BNC27', 'BNC28', 'BNC29', 'BNC30', 'BNC31', 'BNC32', 'BNC33', 'BNC34', 'BNC35', 'BNC36', 'BNC37']


In [5]:
target_list = ['Automobile shop', 'Police station ', 'Fire Station', 'Bank', 'Pawn Shop', 'Pizzeria', 'Quattroki Restaurant', 'High School']
trial_list = [1,2,3]
for i in trial_list:
    for target in target_list:
        if os.path.exists(fp_folder + '/Target_Data/b' + str(i) +'_'+target+'_results.csv')==True:
            os.remove(fp_folder + '/Target_Data/b' + str(i) +'_'+target+'_results.csv')
        if os.path.exists(fp_folder + '/Target_Data/all_'+target+'_results.csv')==True:
            os.remove(fp_folder + '/Target_Data/all_'+target+'_results.csv')

In [6]:
for PID in sub_folders:
    
    # Filepath to each participant folder
    fp_PID = fp_folder + PID
    target_list = ['Automobile shop', 'Police station ', 'Fire Station', 'Bank', 'Pawn Shop', 'Pizzeria', 'Quattroki Restaurant', 'High School']

    for i in range(1,4):
        fp_data = fp_PID +'/Saved_data_'+ PID + '_t' + str(i) + '.csv'

        data = process_data(fp_data)
        for target in target_list:
            data_target = data[data['Target_Name'] == target]
            data_target = data_target.drop_duplicates(subset = ['X','Z','Target_Name'],keep = ('last'))
            data_final = data_target.filter(['X','Z','Target_Name'], axis=1)
            data_final.insert(0,'Participant',PID)
            data_final.insert(1,'Block_num',i)
            if os.path.exists(fp_folder + '/Target_Data/b' + str(i) +'_'+target+'_results.csv')==False:
                data_final.to_csv(fp_folder + '/Target_Data/b' + str(i) +'_'+target+'_results.csv', index=True)
                print('csv created')
            elif os.path.exists(fp_folder + '/Target_Data/b' + str(i) +'_'+target+'_results.csv')==True:
                data_final.to_csv(fp_folder + '/Target_Data/b' + str(i) +'_'+target+'_results.csv', index=True,mode='a', header=False)
            if os.path.exists(fp_folder + '/Target_Data/all_'+target+'_results.csv')==False:
                data_final.to_csv(fp_folder + '/Target_Data/all_'+target+'_results.csv', index=True)
            elif os.path.exists(fp_folder + '/Target_Data/all_'+target+'_results.csv')==True:
                data_final.to_csv(fp_folder + '/Target_Data/all_'+target+'_results.csv', index=True,mode='a', header=False)

csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
csv created
